<a href="https://colab.research.google.com/github/pfyeh1/ner_nyt/blob/main/eda_orgs_nytimes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls "/content/drive/My Drive/_data/"

2019_Gaz_counties_national.txt		  groupby_tests.gsheet
abc_news				  grp_rel_flp.csv
all-geocodes-v2018.xlsx			  idiap
array_csv.csv				  iDIAP_ifcs_12JUN2019.xlsx
bbc					  ifcs
carnegie_pages				  kaggle
carnegie_pdfs				  matches_validated.xlsx
citations				  matches.xlsx
classsifications.csv			  MOCK_DATA2.json
clean_wfa.csv				  mock_sql
clean_wfa.gsheet			  movies
core_metadata.gsheet			  nba
country_list				  ner
cumulative_deaths_county_chloropeth.html  nyt_articles
cumulative_deaths_usa_bubble.html	  okrs
cumulative_deaths_usa.html		  opinRank_Review_Dataset
datacamp				  output
DEC_10_SF1_GCTPH1.US05PR_with_ann.csv	  png_text
DEC_10_SF1_GCTPH1.US05PR_with_ann.gsheet  Pokemon.csv
df_tab.csv				  quora
ebrief					  restaurant
erm					  reviewer_module_data.gsheet
file_renaming				  scrapy
fintel					  state_latlongs.csv
fintel_effort_join			  stock_prices
fuzzy_strings				  swim
Geocodes_USA_with_Counties.csv		  tradecraft_notes
groupby_tables.gsheet			  trump_trudeau_tweets


In [3]:
import os, sys
import pandas as pd
import numpy as np

import string
import re

from tqdm import tqdm

import pickle

#local_python_path = os.path.join(os.environ['HOME'], 'Documents', 'Python')
#os.chdir(local_python_path)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 800)

gpath = os.path.join('/content','drive', 'My Drive', '_data', 'nyt_articles')
os.chdir(gpath)

In [33]:
# infile
infile = "nyt_articles.pkl"

# infile path
infile_path = os.path.join(gpath, "output", infile)

# load dataframe from pkl
df = pd.read_pickle(infile_path)

In [34]:
df.columns

Index(['abstract', 'web_url', 'snippet', 'lead_paragraph', 'print_section',
       'print_page', 'source', 'multimedia', 'keywords', 'pub_date',
       'document_type', 'news_desk', 'section_name', 'type_of_material', '_id',
       'word_count', 'uri', 'headline.main', 'headline.kicker',
       'headline.content_kicker', 'headline.print_headline', 'headline.name',
       'headline.seo', 'headline.sub', 'byline.original', 'byline.person',
       'byline.organization', 'subsection_name'],
      dtype='object')

In [35]:
cols = [
    'uri',
    'web_url',
    'snippet',
    'lead_paragraph',
    'keywords',
    'pub_date',
    'document_type',
    'news_desk',
    'section_name',
  ]

In [36]:
# don't need all the cols
df = df[cols].copy()

In [37]:
df.isnull().sum()

uri               0
web_url           0
snippet           0
lead_paragraph    0
keywords          0
pub_date          0
document_type     0
news_desk         0
section_name      0
dtype: int64

In [38]:
df['pub_date'] = pd.to_datetime(df['pub_date']).dt.strftime('%Y-%m-%d')

In [39]:
df.head()

,uri,web_url,snippet,lead_paragraph,keywords,pub_date,document_type,news_desk,section_name
0,nyt://article/df646720-840a-559b-9019-a38e073d374d,https://www.nytimes.com/2022/10/31/nyregion/rikers-inmate-stabbing-correction-officer.html,"The officer was seriously injured in the attack, which took place in the protective custody unit at Rikers, a union official said.","A city correction officer at the Rikers Island prison complex was seriously injured on Monday when a detainee stabbed him more than a dozen times in the back of the head, the correction officers’ union said in a statement.","[{'name': 'subject', 'value': 'Prison Guards and Corrections Officers', 'rank': 1, 'major': 'N'}, {'name': 'organizations', 'value': 'Rikers Island Prison Complex', 'rank': 2, 'major': 'N'}, {'name': 'subject', 'value': 'Assaults', 'rank': 3, 'major': 'N'}, {'name': 'organizations', 'value': 'Correction Officers Benevolent Assn', 'rank': 4, 'major': 'N'}]",2022-11-01,article,Metro,New York
1,nyt://article/bea5feb0-69e1-53a8-a0d0-eb8028756bfd,https://www.nytimes.com/2022/10/31/sports/baseball/world-series-astros-game-3-phillies.html,"The Houston Astros and the Philadelphia Phillies are tied at one game apiece. The series is scheduled to resume Tuesday, and each remaining game will be pushed back one day.","PHILADELPHIA — Game 3 of the World Series was postponed Monday because of rain, with the Houston Astros and the Philadelphia Phillies tied at one game apiece. The series is scheduled to resume Tuesday, and in place of eliminating the built-in travel day, each game will be pushed back one day.","[{'name': 'subject', 'value': 'Baseball', 'rank': 1, 'major': 'N'}, {'name': 'subject', 'value': 'World Series', 'rank': 2, 'major': 'N'}, {'name': 'subject', 'value': 'Playoff Games', 'rank': 3, 'major': 'N'}, {'name': 'organizations', 'value': 'Houston Astros', 'rank': 4, 'major': 'N'}, {'name': 'organizations', 'value': 'Philadelphia Phillies', 'rank': 5, 'major': 'N'}, {'name': 'persons', 'value': 'McCullers, Lance Jr (1993- )', 'rank': 6, 'major': 'N'}, {'name': 'persons', 'value': 'Syn...",2022-11-01,article,Sports,Sports
2,nyt://article/b54435b9-a1e6-53a5-909a-58142f8ce981,https://www.nytimes.com/2022/10/31/crosswords/daily-puzzle-2022-11-01.html,Make some connections with Bruce Haight’s puzzle.,TUESDAY PUZZLE — Rumor has it that this is the 61st puzzle constructed by Bruce Haight for The Times. That’s a lot of puzzles!,"[{'name': 'subject', 'value': 'Crossword Puzzles', 'rank': 1, 'major': 'N'}, {'name': 'persons', 'value': 'Haight, Bruce', 'rank': 2, 'major': 'N'}]",2022-11-01,article,Games,Crosswords & Games
3,nyt://article/e860a43e-23b4-53d5-b1b7-9c87012e6e58,https://www.nytimes.com/2022/10/31/us/steven-pankey-murder-kidnapping-id.html,"The long-unsolved kidnapping and killing of Jonelle Matthews haunted the community of Greeley, about 50 miles north of Denver.","A Colorado jury on Monday convicted an Idaho man of murder and kidnapping in the 1984 disappearance and death of a 12-year-old girl who was killed days before Christmas, a case that went unsolved for decades and haunted the tight-knit community of Greeley, Colo., where she lived.","[{'name': 'subject', 'value': 'Murders, Attempted Murders and Homicides', 'rank': 1, 'major': 'N'}, {'name': 'subject', 'value': 'Kidnapping and Hostages', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Matthews, Jonelle', 'rank': 3, 'major': 'N'}, {'name': 'persons', 'value': 'Pankey, Steven D (1951- )', 'rank': 4, 'major': 'N'}, {'name': 'glocations', 'value': 'Greeley (Colo)', 'rank': 5, 'major': 'N'}, {'name': 'glocations', 'value': 'Idaho', 'rank': 6, 'major': 'N'}, {'name': 's...",2022-11-01,article,Express,U.S.
4,nyt://video/bb3260bb-b2a2-5e3d-96be-4a9526b1667f,https://www.nytimes.com/video/us/politics/100000008610372/pelosi-attack-suspect-charges.html,"David DePape, who is accused of breaking into the San Francisco home of Speaker Nancy Pelosi, was charged with attempting to kidnap Ms. Pel

In [40]:
print(df['pub_date'].max())
print(df['pub_date'].min())

2023-10-22
2022-11-01


In [41]:
# filters
date = (df['pub_date']>='2023-10-07')
section = (df['section_name']=='World')
document_type = (df['document_type']=='article')

df1 = df.loc[date & section & document_type].copy()

In [42]:
# define the key-value pair you want to filter for
target_key = 'glocations'
target_value = 'Israel'

df2 = df1[df1['keywords'].apply(lambda x: any(entry['name'] == target_key and entry['value'] == target_value for entry in x))].copy()


In [43]:
print(len(df1))
print(len(df2))
df2.sample(n=3)

301
126


,uri,web_url,snippet,lead_paragraph,keywords,pub_date,document_type,news_desk,section_name
45388,nyt://article/a7235913-253a-517e-b801-5d42418d725e,https://www.nytimes.com/2023/10/18/world/middleeast/gaza-hospital-deaths-aftermath.html,"“We’ve never lived through a war this intense,” said a Palestinian journalist who captured the aftermath of the blast in Gaza that killed hundreds.","Sameh al-Jaroosha was sitting on the edge of the grassy courtyard of the Ahli Arab Hospital in Gaza City on Tuesday night, talking to a new acquaintance who was among the thousands of Palestinian residents of the city seeking refuge there in hopes that it would be safer than staying at home.","[{'name': 'subject', 'value': 'Hospitals', 'rank': 1, 'major': 'N'}, {'name': 'subject', 'value': 'Israel-Gaza War (2023- )', 'rank': 2, 'major': 'N'}, {'name': 'organizations', 'value': 'Al Ahli Arab Hospital (Gaza Strip)', 'rank': 3, 'major': 'N'}, {'name': 'subject', 'value': 'Civilian Casualties', 'rank': 4, 'major': 'N'}, {'name': 'organizations', 'value': 'Hamas', 'rank': 5, 'major': 'N'}, {'name': 'glocations', 'value': 'Gaza Strip', 'rank': 6, 'major': 'N'}, {'name': 'subject', 'valu...",2023-10-18,article,Foreign,World
44288,nyt://article/ea6d081c-9376-5bf6-ab4d-d71a12777bdd,https://www.nytimes.com/2023/10/10/world/middleeast/kfar-azza-gaza-hamas-attack.html,"Kfar Azza, less than two miles from Gaza, was one of the villages overrun by Palestinian gunmen on Saturday.","The village of Kfar Azza looked normal from a distance — tidy terraces of one-story beige houses. I walked past the village dining hall, kindergarten and culture center, and turned left.","[{'name': 'glocations', 'value': 'Israel', 'rank': 1, 'major': 'N'}, {'name': 'glocations', 'value': 'Gaza Strip', 'rank': 2, 'major': 'N'}, {'name': 'organizations', 'value': 'Hamas', 'rank': 3, 'major': 'N'}, {'name': 'subject', 'value': 'Israel-Gaza War (2023- )', 'rank': 4, 'major': 'N'}]",2023-10-10,article,Foreign,World
45161,nyt://article/f72928f5-d53c-5cb5-805a-d5a421a02c7c,https://www.nytimes.com/2023/10/16/world/middleeast/gaza-invasion-israel-cellphone-data.html,"The Israeli military says the tracking and the call to evacuate are to preserve as many civilian lives as possible during an expected ground invasion, even as deadly airstrikes continue.","In a windowless control room on an army base in southern Israel, five soldiers monitored the displacement of hundreds of thousands of Gazans on a massive computer screen.","[{'name': 'subject', 'value': 'Israel-Gaza War (2023- )', 'rank': 1, 'major': 'N'}, {'name': 'subject', 'value': 'Defense and Military Forces', 'rank': 2, 'major': 'N'}, {'name': 'subject', 'value': 'War and Armed Conflicts', 'rank': 3, 'major': 'N'}, {'name': 'subject', 'value': 'International Relations', 'rank': 4, 'major': 'N'}, {'name': 'glocations', 'value': 'Gaza Strip', 'rank': 5, 'major': 'N'}, {'name': 'glocations', 'value': 'Israel', 'rank': 6, 'major': 'N'}, {'name': 'organization...",2023-10-16,article,Foreign,World


In [44]:
# peek at first record of keywords field
df2.loc[df2['uri']=='nyt://article/ccd81853-4659-5a9d-b2e0-53489a78db39']['keywords']

44747    [{'name': 'subject', 'value': 'Israel-Gaza War (2023- )', 'rank': 1, 'major': 'N'}, {'name': 'glocations', 'value': 'Israel', 'rank': 2, 'major': 'N'}, {'name': 'subject', 'value': 'Defense and Military Forces', 'rank': 3, 'major': 'N'}, {'name': 'glocations', 'value': 'Gaza Strip', 'rank': 4, 'major': 'N'}, {'name': 'subject', 'value': 'Palestinians', 'rank': 5, 'major': 'N'}, {'name': 'subject', 'value': 'Evacuations and Evacuees', 'rank': 6, 'major': 'N'}, {'name': 'subject', 'value': 'Re...
Name: keywords, dtype: object

In [45]:
# turn dictionary in list into just list of vlaues
#df2['keywords_'] = df2['keywords'].apply(lambda x: [i['value'] for i in x])

In [46]:
import spacy.cli
spacy.cli.download("en_core_web_md")
import spacy
import en_core_web_md
#from spacy.pipeline import EntityRuler
nlp = spacy.load("en_core_web_md")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [47]:
ents = [
    'Hezbollah',
    'Hamas']

In [48]:
# initaliz empty list and add onewrd_nse_terrs to ent_patrns
ent_patrns = []

for ent in ents:
  ent_ = {"label":"ORG", "pattern":[{"LOWER":ent.lower()}]}
  ent_patrns.append(ent_)

In [64]:
ent_patrns

[{'label': 'ORG', 'pattern': [{'LOWER': 'hezbollah'}]},
 {'label': 'ORG', 'pattern': [{'LOWER': 'hamas'}]}]

In [49]:
# initialize EntityRuler
#ruler = EntityRuler(nlp, overwrite_ents = True, validate = True)

# initialize Entity Ruler
ruler = nlp.add_pipe("entity_ruler")

In [50]:
# add patterns to ruler
ruler.add_patterns(ent_patrns)

In [65]:
# some testing
doc = nlp("Hamas wants to attack aliens")

In [66]:
for ent in doc.ents:
  print(ent.text, ent.label_)

Hamas ORG


In [67]:
# add ruler to pipeline
#nlp.add_pipe(ruler, before = "ner")

In [74]:
# function
def extract_named_ents3(text):
  "use spacy to extract GPEs"
  ent_labels = ["GPE", "NORP", "ORG"]
  doc = nlp(text)
  items = [(x.text, x.label_) for x in doc.ents if x.label_ in ent_labels] # use to get NORP and GPE
  items_counter = Counter(items)#.most_common(10)
  return items_counter

def extract_named_ents2(text):
  "use spacy to extract GPEs"
  ent_labels = ["GPE", "NORP", "ORG"]
  doc = nlp(text)
  items = [(x.text, x.label_) for x in doc.ents if x.label_ in ent_labels] # use to get specified labels
  #items = [(x.text, x.label_) for x in doc.ents] # use to get all ents
  return items

In [70]:
# instaniate tqdm
tqdm.pandas()

In [75]:
# apply extraction function
df2['ents_extracted'] = df2['lead_paragraph'].str.lower().progress_apply(extract_named_ents3)

100%|██████████| 126/126 [00:02<00:00, 59.73it/s]


In [79]:
df2[['lead_paragraph', 'keywords', 'ents_extracted']].sample(n=3)

,lead_paragraph,keywords,ents_extracted
44631,"As Israeli warplanes pound Gaza, killing more than 1,500 people and crushing buildings in response to last weekend’s unprecedented attack by Hamas, Israel’s leadership has repeatedly urged civilians to flee the territory while they can.","[{'name': 'subject', 'value': 'Israel-Gaza War (2023- )', 'rank': 1, 'major': 'N'}, {'name': 'glocations', 'value': 'Israel', 'rank': 2, 'major': 'N'}, {'name': 'glocations', 'value': 'Gaza Strip', 'rank': 3, 'major': 'N'}, {'name': 'subject', 'value': 'Palestinians', 'rank': 4, 'major': 'N'}, {'name': 'organizations', 'value': 'Hamas', 'rank': 5, 'major': 'N'}, {'name': 'glocations', 'value': 'Egypt', 'rank': 6, 'major': 'N'}, {'name': 'subject', 'value': 'Refugees and Displaced Persons', '...","{('israeli', 'NORP'): 1, ('pound gaza', 'GPE'): 1, ('hamas', 'GPE'): 1, ('israel', 'GPE'): 1}"
44136,"A former spokesman for Israel’s military gave a blunt assessment of the situation in Israel on Sunday, warning that it would take hours or longer to reclaim all the towns in southern Israel where the military was still battling Hamas militants.","[{'name': 'subject', 'value': 'Israel-Gaza War (2023- )', 'rank': 1, 'major': 'N'}, {'name': 'glocations', 'value': 'Israel', 'rank': 2, 'major': 'N'}, {'name': 'glocations', 'value': 'Gaza Strip', 'rank': 3, 'major': 'N'}, {'name': 'organizations', 'value': 'Hamas', 'rank': 4, 'major': 'N'}]","{('israel', 'GPE'): 3, ('hamas', 'ORG'): 1}"
45198,"A young Palestinian man from East Jerusalem, Soheib Muhammad Abu Amar, is missing and believed to have been taken hostage by Hamas and ferried into Gaza, his family said.","[{'name': 'subject', 'value': 'Israel-Gaza War (2023- )', 'rank': 1, 'major': 'N'}, {'name': 'subject', 'value': 'Palestinians', 'rank': 2, 'major': 'N'}, {'name': 'glocations', 'value': 'Gaza Strip', 'rank': 3, 'major': 'N'}, {'name': 'glocations', 'value': 'Israel', 'rank': 4, 'major': 'N'}, {'name': 'organizations', 'value': 'Hamas', 'rank': 5, 'major': 'N'}]","{('palestinian', 'NORP'): 1, ('east jerusalem', 'GPE'): 1, ('soheib', 'GPE'): 1, ('hamas', 'ORG'): 1, ('gaza', 'GPE'): 1}"


In [87]:
df2.loc[df2['lead_paragraph'].str.contains(r'hezbollah', case = False, na = False)]

,uri,web_url,snippet,lead_paragraph,keywords,pub_date,document_type,news_desk,section_name,ents_extracted
44366,nyt://article/ca20f735-f773-5840-a5a4-0ad90d49f2ec,https://www.nytimes.com/2023/10/10/world/middleeast/israel-syria-lebanon-shelling.html,Continued attacks on Israel’s northern border have deepened unease over the possibility that the conflict — already the broadest invasion in 50 years — could spread.,"Shelling sounded out along Israel’s volatile northern border with Lebanon for a third consecutive day Tuesday, stoking fears on both sides of a repeat of 2006, when Israel fought a bloody monthlong war with Hezbollah, the Shiite group committed to the destruction of the Jewish state.","[{'name': 'subject', 'value': 'Israel-Gaza War (2023- )', 'rank': 1, 'major': 'N'}, {'name': 'organizations', 'value': 'Hezbollah', 'rank': 2, 'major': 'N'}, {'name': 'glocations', 'value': 'Israel', 'rank': 3, 'major': 'N'}, {'name': 'glocations', 'value': 'Lebanon', 'rank': 4, 'major': 'N'}, {'name': 'glocations', 'value': 'Syria', 'rank': 5, 'major': 'N'}]",2023-10-10,article,Foreign,World,"{('israel', 'GPE'): 2, ('lebanon', 'GPE'): 1, ('hezbollah', 'ORG'): 1, ('the shiite group committed', 'ORG'): 1, ('jewish', 'NORP'): 1}"
45557,nyt://article/818767e8-69f0-550e-a113-421ef523ec46,https://www.nytimes.com/2023/10/19/world/middleeast/hezbollah-lebanon-israel-explained.html,"For decades, Hezbollah has been a significant force in Lebanon, wielding both political power and military strength — all while engaging in tit-for-tat skirmishes with Israel.","As Israel prepares a possible ground invasion of Gaza to destroy Hamas, the Palestinian militant group that controls the enclave, many are now turning to see what action Hezbollah, the armed group on Israel’s northern border, will take. For decades, Hezbollah has been a significant force in Lebanon, wielding both political power and military strength — all while engaging in tit-for-tat skirmishes with its southern enemy, Israel.","[{'name': 'subject', 'value': 'Israel-Gaza War (2023- )', 'rank': 1, 'major': 'N'}, {'name': 'subject', 'value': 'War and Armed Conflicts', 'rank': 2, 'major': 'N'}, {'name': 'organizations', 'value': 'Hezbollah', 'rank': 3, 'major': 'N'}, {'name': 'organizations', 'value': 'Hamas', 'rank': 4, 'major': 'N'}, {'name': 'glocations', 'value': 'Lebanon', 'rank': 5, 'major': 'N'}, {'name': 'glocations', 'value': 'Israel', 'rank': 6, 'major': 'N'}]",2023-10-19,article,Foreign,World,"{('israel', 'GPE'): 3, ('gaza', 'GPE'): 1, ('hamas', 'GPE'): 1, ('palestinian', 'NORP'): 1, ('hezbollah', 'ORG'): 2, ('lebanon', 'GPE'): 1}"


In [89]:
df2.loc[df2['lead_paragraph'].str.contains(r'hezbollah', case = False, na = False)][['lead_paragraph', 'ents_extracted']].head()

,lead_paragraph,ents_extracted
44366,"Shelling sounded out along Israel’s volatile northern border with Lebanon for a third consecutive day Tuesday, stoking fears on both sides of a repeat of 2006, when Israel fought a bloody monthlong war with Hezbollah, the Shiite group committed to the destruction of the Jewish state.","{('israel', 'GPE'): 2, ('lebanon', 'GPE'): 1, ('hezbollah', 'ORG'): 1, ('the shiite group committed', 'ORG'): 1, ('jewish', 'NORP'): 1}"
45557,"As Israel prepares a possible ground invasion of Gaza to destroy Hamas, the Palestinian militant group that controls the enclave, many are now turning to see what action Hezbollah, the armed group on Israel’s northern border, will take. For decades, Hezbollah has been a significant force in Lebanon, wielding both political power and military strength — all while engaging in tit-for-tat skirmishes with its southern enemy, Israel.","{('israel', 'GPE'): 3, ('gaza', 'GPE'): 1, ('hamas', 'GPE'): 1, ('palestinian', 'NORP'): 1, ('hezbollah', 'ORG'): 2, ('lebanon', 'GPE'): 1}"


In [ ]:
# dump list into a pickle file to load later as an option
pkl_file = "processed_article.pkl"
reuse_path = os.path.join(gpath, "output", pkl_file)

In [ ]:
# save pkl
df.to_pickle(reuse_path)

# output csv
df.to_csv(os.path.join(gpath, "output")+'/processed_articles.csv', index = None)

In [ ]:
# load from resue path
df = pd.read_pickle(reuse_path)

In [ ]:
def percent_ents(ents):
  new_ents=[]
  total = sum([ent[1] for ent in ents])
  for ent in ents:
    percent=(ent[1])/total
    new_ents.append([ent[0]]+[percent])
  return new_ents

def percent_ents2(ents):
  new_ents = []
  for ent in ents:
    perc_total = round((ent[1]/sum([ent[1] for ent in ents])),2)
    new_ents.append([ent[0]]+[perc_total])
  return new_ents


def limit_ents_orig(ents):
  for i in ents:
    if i[1] >=2:
      return [i[0][0] for i in ents]

def limit_ents(ents, thresh):
  ents_limited = [] # initialize empty list
  for ent in ents:
    if ent[1] >= thresh:
      ents_limited.append(ent[0])
  return ents_limited


def limit_ents2(ents):
  for i in ents:
    if i[1] >=.5:
      return [i[0][0] for i in ents]

def clean_ents_old(ents):
  new_ents = []
  for ent in ents:
    changed_ents = [ent_dict.get(k,k) for k in ent[0]]
    new_ents.append(changed_ents)
  return new_ents

def clean_ents(ents):
  new_ents = []
  for ent in ents:
    changed_ents = [ent_dict.get(k,k) for k in ent[0]]
    #changed_ents = ent[0][0]
    new_ents.append(changed_ents[:1]+[ent[1]])
  return new_ents

def ents_consolidate(ents):
  grp = {}
  for inner in ents:
    key = (inner[0]) # tuple is immutable and can be key
    grp.setdefault(key,0) # create key if needed with value 0
    grp[key]+=inner[1] # add value
  list_out = [[k,grp[k]] for k in grp] # reassemble from dictionary
  return list_out


In [ ]:
# use function to calculate % of ents
df['percent_ents'] = df['ents_extracted'].progress_apply(percent_ents)

100%|██████████| 4542/4542 [00:00<00:00, 218351.68it/s]


In [ ]:
# use function to resolve ents (e.g., China, Chinese)
df['ents_cleaned'] = df['percent_ents'].progress_apply(clean_ents)

100%|██████████| 4542/4542 [00:00<00:00, 146964.20it/s]


In [ ]:
# use function to consolidate ents that were extracted multiple times
df['ents_consolidated'] = df['ents_cleaned'].progress_apply(ents_consolidate)

100%|██████████| 4542/4542 [00:00<00:00, 160788.38it/s]


In [ ]:
df['ents_limited'] = df['ents_consolidated'].progress_apply(limit_ents, thresh = .20)

100%|██████████| 4542/4542 [00:00<00:00, 232601.51it/s]


In [ ]:
# check calcs
print(df.iloc[3]['ents_extracted'])
print(df.iloc[3]['percent_ents'])
print(df.iloc[3]['ents_cleaned'])
print(df.iloc[3]['ents_consolidated'])
print(df.iloc[3]['ents_limited'])

[(('american', 'NORP'), 3), (('baghdad', 'GPE'), 2), (('pro-iranian', 'NORP'), 1), (('iraqi', 'NORP'), 1), (('iraq', 'GPE'), 1)]
[[('american', 'NORP'), 0.375], [('baghdad', 'GPE'), 0.25], [('pro-iranian', 'NORP'), 0.125], [('iraqi', 'NORP'), 0.125], [('iraq', 'GPE'), 0.125]]
[['united states', 0.375], ['iraq', 0.25], ['pro-iranian', 0.125], ['iraq', 0.125], ['iraq', 0.125]]
[['united states', 0.375], ['iraq', 0.5], ['pro-iranian', 0.125]]
['united states', 'iraq']


In [ ]:
df['tgt_ents'] = df['ents_limited'].apply(extract_tgt_ents)

In [ ]:
# check calcs
n = 748
print(df.iloc[n]['ents_extracted'])
print(df.iloc[n]['percent_ents'])
print(df.iloc[n]['ents_cleaned'])
print(df.iloc[n]['ents_consolidated'])
print(df.iloc[n]['ents_limited'])
print(df.iloc[n]['tgt_ents'])

[(('hong kong', 'GPE'), 2), (('china', 'GPE'), 2), (('swedish', 'NORP'), 2), (('chinese', 'NORP'), 2)]
[[('hong kong', 'GPE'), 0.25], [('china', 'GPE'), 0.25], [('swedish', 'NORP'), 0.25], [('chinese', 'NORP'), 0.25]]
[['hong kong', 0.25], ['china', 0.25], ['sweden', 0.25], ['china', 0.25]]
[['hong kong', 0.25], ['china', 0.5], ['sweden', 0.25]]
['hong kong', 'china', 'sweden']
['china']


In [ ]:
# make boolean col; somewhat of unecessary but at least it's there
df['tgt_ents_contain_china'] = df['tgt_ents'].apply(lambda x: any(i == 'china' for i in x))

In [ ]:
# filter for contains china = True
df1 = df.loc[df['tgt_ents_contain_china']==True].copy()

In [ ]:
print("No. of records where tgt ents contain China: ",len(df1))

No. of records where tgt ents contain China:  460


In [ ]:
# funcs to help with calcs

def compare(lst, s):
  if s in lst:
    return(True)
  else:
    return(False)

def calcs(df):
  # number of rows where China is in tgt_ents_contain_china and keywords
  df1['kwds_contain_China'] = df1['keywords_'].apply(compare, s = 'China')
  df2 = df1.loc[df1['kwds_contain_China']==True].copy()
  df3 = df1.loc[df1['kwds_contain_China']==False].copy()
  print("No. of records where tgt ents contain China: ",len(df1))
  print("No. of records where China is in keywords and tgt_extracted_ents: ", len(df2))
  print("No. of records where China is not in keywords and tgt_extracted_ents: ", len(df3))
  print("% increase in China-related production not based on country metadata: ", 100*round((len(df1)-len(df2))/len(df1),2))

In [ ]:
calcs(df1)

No. of records where tgt ents contain China:  460
No. of records where China is in keywords and tgt_extracted_ents:  349
No. of records where China is not in keywords and tgt_extracted_ents:  111
% increase in China-related production not based on country metadata:  24.0


In [ ]:
df1.head()

,_id,word_count,pub_date,lead_paragraph,snippet,keywords,merged_text,keywords_,ents_extracted,percent_ents,ents_cleaned,ents_consolidated,ents_limited,tgt_ents,tgt_ents_contain_china
412,nyt://article/51dbcda3-c99e-54b0-b531-8f08852dd868,815,2020-01-04T13:30:04+0000,"BEIJING — The Chinese government abruptly replaced its top representative in Hong Kong on Saturday evening, installing a senior Communist Party official with a record of difficult assignments in inland provinces that involved working closely with the security services.","After months of protests and an electoral defeat for pro-Beijing political parties, an official with a record of difficult assignments is on the way.","[{'name': 'glocations', 'value': 'Hong Kong', 'rank': 1, 'major': 'N'}, {'name': 'glocations', 'value': 'China', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Wang Zhimin', 'rank': 3, 'major': 'N'}, {'name': 'persons', 'value': 'Luo Huining', 'rank': 4, 'major': 'N'}, {'name': 'subject', 'value': 'Politics and Government', 'rank': 5, 'major': 'N'}, {'name': 'organizations', 'value': 'Communist Party of China', 'rank': 6, 'major': 'N'}, {'name': 'subject', 'value': 'Appointments and...","BEIJING — The Chinese government abruptly replaced its top representative in Hong Kong on Saturday evening, installing a senior Communist Party official with a record of difficult assignments in inland provinces that involved working closely with the security services.After months of protests and an electoral defeat for pro-Beijing political parties, an official with a record of difficult assignments is on the way.","[Hong Kong, China, Wang Zhimin, Luo Huining, Politics and Government, Communist Party of China, Appointments and Executive Changes, Hong Kong Protests (2019)]","[((beijing, GPE), 1), ((chinese, NORP), 1), ((hong kong, GPE), 1)]","[[(beijing, GPE), 0.3333333333333333], [(chinese, NORP), 0.3333333333333333], [(hong kong, GPE), 0.3333333333333333]]","[[china, 0.3333333333333333], [china, 0.3333333333333333], [hong kong, 0.3333333333333333]]","[[china, 0.6666666666666666], [hong kong, 0.3333333333333333]]","[china, hong kong]",[china],True
609,nyt://article/1d368b31-8145-5fa7-b313-ca3b3e742cc9,985,2020-01-06T17:14:52+0000,"BEIJING — For days, Li Bin had what felt like a cold, with a high fever of between 102 and 105, and he could not understand why he wasn’t getting better.",Beijing is racing to identify a new illness that has sickened 59 people as it tries to calm a nervous public.,"[{'name': 'subject', 'value': 'Respiratory Diseases', 'rank': 1, 'major': 'N'}, {'name': 'subject', 'value': 'Epidemics', 'rank': 2, 'major': 'N'}, {'name': 'subject', 'value': 'Pneumonia', 'rank': 3, 'major': 'N'}, {'name': 'glocations', 'value': 'China', 'rank': 4, 'major': 'N'}, {'name': 'glocations', 'value': 'Hong Kong', 'rank': 5, 'major': 'N'}, {'name': 'glocations', 'value': 'Singapore', 'rank': 6, 'major': 'N'}, {'name': 'glocations', 'value': 'Wuhan (China)', 'rank': 7, 'major': 'N'}]","BEIJING — For days, Li Bin had what felt like a cold, with a high fever of between 102 and 105, and he could not understand why he wasn’t getting better. Beijing is racing to identify a new illness that has sickened 59 people as it tries to calm a nervous public.","[Respiratory Diseases, Epidemics, Pneumonia, China, Hong Kong, Singapore, Wuhan (China)]","[((beijing, GPE), 2)]","[[(beijing, GPE), 1.0]]","[[china, 1.0]]","[[china, 1.0]]",[china],[china],True
829,nyt://article/4512f341-2edf-5dfb-a6e3-64c1c2aff950,993,2020-01-08T01:05:30+0000,"The lurid court testimony riveted Canada: A Chinese immigrant shot dead his brash, womanizing, millionaire relative at his $8 million hillside mansion in Vancouver, and then chopped up the body into 108 pieces before taking a long nap.","After a trial that took more than two years, Zhao Li, who killed his millionaire cousin and then chopped up the body, was found guilty of manslaughter.","[{'name': 'subject', 'value': 'Murders, Attempted Mu

In [ ]:
# check for target entities
#df['ents_check'] = df['ents_limited'].apply(ent_check)

In [ ]:
# count target entities
#df["ents_count"]  = df['ents3'].apply(ent_count)

In [ ]:
# replace None object with False
#df['ents_check'].fillna(False, inplace = True)

In [ ]:
# extract ents from ents3
#df['all_ents'] = df['ents3'].apply(extract_all_ents)

In [ ]:
# lower case string
#df['all_ents'] = df['all_ents'].apply(lambda x: [i.lower() for i in x])

In [ ]:
# make dict to remap elements

#ent_dict = {
#            "chinese":"china",
#            "beijing":"china"
#}

# map elements in list using ent dict
#df['all_ents'] = df['all_ents'].apply(lambda x:[ent_dict.get(k,k) for k in x])

In [ ]:
# get unique values in list
#df['all_ents'] = df['all_ents'].map(set).map(list)

In [ ]:
#df.head()

In [ ]:
# check if element in list matchces condition
#df['contains_china'] = df['all_ents'].apply(lambda x: any(i == 'china' for i in x))

#all(i >= 30 for i in my_list1)
#any([target in x[0] for x in entities]):

In [ ]:
#df['tgt_ents'] = df['all_ents'].apply(extract_tgt_ents)

In [ ]:
#df[df['contains_china']].head()

In [ ]:
#df['contains_china'] = df['all_ents'].apply(lambda x: any(i == 'china' for i in x))

In [ ]:
#len(df1)

In [ ]:
#df1.sample(n=5)

In [ ]:
# convert pubdate to datetime object
#df1['pub_date'] = pd.to_datetime(df1['pub_date'])

In [ ]:
# lower case all strings in tgt_ents col
#df1['tgt_ents'] = df1['tgt_ents'].apply(lambda x: [i.lower() for i in x])

In [ ]:
#df1.dtypes

In [ ]:
#import seaborn as sns

In [ ]:
#df_explode = df1.explode('tgt_ents')

In [ ]:
#df_explode.head()

In [ ]:
#plt.figure(figsize = (10,8))
#df_explode['tgt_ents'].value_counts().plot(kind = 'bar')
#plt.title("Value Counts of Entities")
#plt.show()

In [ ]:
# extract elements from tgt_ents column into separate dataframe
#df_ents = pd.DataFrame(df1['tgt_ents'].values.tolist(), df1.index).add_prefix('ent_')

In [ ]:
# concat df1 and df_ents
#df_final = pd.concat([df1, df_ents], axis=1)

In [ ]:
#df_final.head()

In [ ]:
#plt.figure(figsize = (10,8))
#df_final['ent_0'].value_counts().plot(kind = 'bar')
#plt.title("Value Counts of Entities in Column Ent0")
#plt.show()